for dataset :
https://www.kaggle.com/datasets/algord/fake-news

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences


In [ ]:
df = pd.read_csv("FakeNewsNet.csv")
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [ ]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23196 entries, 0 to 23195
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          23196 non-null  object
 1   news_url       22866 non-null  object
 2   source_domain  22866 non-null  object
 3   tweet_num      23196 non-null  int64 
 4   real           23196 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 906.2+ KB


In [ ]:
df = df.dropna()
df.shape

(22866, 5)

In [ ]:
y = df["real"]
y.value_counts()

1    17371
0     5495
Name: real, dtype: int64

In [ ]:
df.drop("real", axis = 1, inplace =True)
data_copy = df.copy()
data_copy.head()

<ipython-input-9-afa58bba4071>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("real", axis = 1, inplace =True)


,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38


In [ ]:
x = df["title"]
x.shape

(22866,)

In [ ]:
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
ps = PorterStemmer()
import re
corpus = []
for i in x:
  a = re.sub("[^a-zA-Z ]","", i)
  a = a.lower()
  a = a.split()

  a =" ".join([ps.stem(w) for w in a if not w in stopwords.words('english')])
  corpus.append(a)


In [ ]:
corpus[0:6]

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill costar hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus']

In [ ]:
voc_size=20000
one_hot_data = [one_hot(sen, voc_size,) for sen in corpus]
one_hot_data[0:5]

[[5417, 11663, 2895, 10365, 13133, 8652, 17840, 4091, 4172, 657],
 [9051, 3128, 10894, 7765, 15237, 2828, 5843],
 [8345,
  12188,
  4022,
  12844,
  11617,
  15503,
  19961,
  17228,
  5032,
  1948,
  19597,
  14401,
  9202,
  19865],
 [1704, 13439, 7698, 12184, 16653, 3893, 19381, 4159],
 [8440,
  12815,
  11108,
  8954,
  11261,
  12801,
  9968,
  13002,
  3251,
  15867,
  19641,
  7732,
  14668]]

In [ ]:
sen_len = 25
pad_data = pad_sequences(one_hot_data, padding = "pre", maxlen=sen_len)
pad_data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,  5417, 11663,  2895,
       10365, 13133,  8652, 17840,  4091,  4172,   657], dtype=int32)

In [ ]:
pad_data = np.array(pad_data)
y = np.array(y)
pad_data.shape, y.shape

((22866, 25), (22866,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad_data, y, test_size=0.23, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((17606, 25), (5260, 25), (17606,), (5260,))

In [ ]:
dim = 40
model = Sequential()
model.add(Embedding(voc_size , dim, input_length= sen_len ))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1, activation = "sigmoid"))
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics= "accuracy")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 40)            800000    
                                                                 
 dropout (Dropout)           (None, 25, 40)            0         
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 913001 (3.48 MB)
Trainable params: 913001 (3.48 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [ ]:
model.fit(X_train,y_train, epochs =10, validation_data =(X_test,y_test), batch_size =32)

Epoch 1/10
551/551 [==============================] - 23s 30ms/step - loss: 0.4484 - accuracy: 0.8046 - val_loss: 0.3756 - val_accuracy: 0.8411
Epoch 2/10
551/551 [==============================] - 7s 13ms/step - loss: 0.3296 - accuracy: 0.8598 - val_loss: 0.4255 - val_accuracy: 0.8314
Epoch 3/10
551/551 [==============================] - 6s 10ms/step - loss: 0.2787 - accuracy: 0.8853 - val_loss: 0.3966 - val_accuracy: 0.8319
Epoch 4/10
551/551 [==============================] - 5s 9ms/step - loss: 0.2394 - accuracy: 0.9042 - val_loss: 0.4174 - val_accuracy: 0.8274
Epoch 5/10
551/551 [==============================] - 7s 12ms/step - loss: 0.2069 - accuracy: 0.9172 - val_loss: 0.4431 - val_accuracy: 0.8234
Epoch 6/10
551/551 [==============================] - 5s 9ms/step - loss: 0.1846 - accuracy: 0.9251 - val_loss: 0.5246 - val_accuracy: 0.8156
Epoch 7/10
551/551 [==============================] - 6s 10ms/step - loss: 0.1616 - accuracy: 0.9347 - val_loss: 0.6189 - val_accuracy: 0.8171


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
pred = model.predict(X_test)
pred

165/165 [==============================] - 1s 3ms/step


array([[0.0715754 ],
       [0.9985973 ],
       [0.99886394],
       ...,
       [0.02114315],
       [0.99999154],
       [0.3600555 ]], dtype=float32)

In [ ]:
# import tensorflow as tf
# pred = tf.argmax(pred, axis=1)
# pred

In [ ]:
new_pred = []
for elements in pred:
  if elements[0] >= 0.5:
    new_pred.append([1])
  else:
    new_pred.append([0])

In [ ]:
confusion_matrix(y_test,new_pred)

array([[ 730,  542],
       [ 489, 3499]])

In [ ]:
accuracy_score(new_pred,y_test)

0.8039923954372623